In [1]:
!pip install -q -U google-generativeai
!pip install -q -U --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [3]:
from datetime import datetime, timedelta
from dotenv import load_dotenv
import PIL.Image
import pytz
import json
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.generativeai as genai

import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

load_dotenv()

API_KEY = str(os.environ.get("API_KEY"))
SCOPES = ["https://www.googleapis.com/auth/calendar"]

def configurar_repeticao(dias_repeticao):
    if dias_repeticao == 1:
        return ["MO"]
    elif dias_repeticao == 2:
        return ["MO", "TH"]
    elif dias_repeticao == 3:
        return ["MO", "WE", "FR"]
    elif dias_repeticao == 4:
        return ["MO", "WE", "FR", "SA"]
    elif dias_repeticao == 5:
        return ["MO", "TU", "WE", "FR", "SA"]
    elif dias_repeticao == 6:
        return ["MO", "TU", "WE", "TH", "FR", "SA"]
    elif dias_repeticao == 7:
        return ["MO", "TU", "WE", "TH", "FR", "SA", "SU"]
    else:
        return []

def main(dicionario):
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("calendar", "v3", credentials=creds)

        # Definir detalhes da tarefa
        titulo_tarefa = f"Regar {dicionario['Nome comum']}"
        descricao_tarefa = f"Regar planta {dicionario['Nome comum']}"
        hoje = datetime.now(pytz.timezone('America/Sao_Paulo'))
        # Agendar para o próximo dia às 18h
        proximo_dia = hoje + timedelta(days=1)
        data_hora_inicio = datetime(proximo_dia.year, proximo_dia.month, proximo_dia.day, 18, 0, 0, tzinfo=pytz.timezone('America/Sao_Paulo'))
        data_hora_fim = data_hora_inicio + timedelta(minutes=15) # Adicionando 15 minutos
        dias_repeticao = dicionario.get('Frequência de rega (semanal)', 1) # Valor padrão 1 se 'rega' não estiver presente no dicionário
         
        dias = configurar_repeticao(dias_repeticao)
        recorrencia = f'RRULE:FREQ=WEEKLY;BYDAY=' + ','.join(dias)

        corpo_evento = {
            'summary': titulo_tarefa,
            'description': descricao_tarefa,
            'start': {
                'dateTime': data_hora_inicio.isoformat(),
                'timeZone': 'America/Sao_Paulo' # Seu fuso horário
            },
            'end': {
                'dateTime': data_hora_fim.isoformat(),
                'timeZone': 'America/Sao_Paulo' # Seu fuso horário
            },
            'recurrence': [recorrencia] # Convertendo para lista
        }

        service.events().insert(calendarId='primary', body=corpo_evento).execute()

    except HttpError as error:
        print(f"An error occurred: {error}")
         
def selecionar_arquivo():
    root = tk.Tk()
    root.withdraw() # Esconde a janela principal do tkinter

    # Abre o navegador de arquivos e retorna o caminho do arquivo selecionado
    arquivo_selecionado = filedialog.askopenfilename()

    return arquivo_selecionado

def popup_opcoes(dic):
    root = tk.Tk()
    root.withdraw() # Esconde a janela principal do tkinter

    texto_dicionario = formatar_dicionario(dic)

    resposta = messagebox.askquestion("Opções", f"Deseja criar uma rotina no seu Google Calendar?\n\n{texto_dicionario}")

    if resposta == 'yes':
        return True
    else:
        return False

def configura_json(resposta):
    resposta = (resposta.text).replace("```json", "")
    resposta = resposta.replace("```", "")
    resposta = json.loads(resposta)
     
    resposta['Frequência de rega (semanal)'] = int(resposta['Frequência de rega (semanal)'])
    resposta['Frequência de fertilização (mensal)'] = int(resposta['Frequência de fertilização (mensal)'])
     
    return resposta

def formatar_dicionario(dic):
    texto_formatado = ""
    for chave, valor in dic.items():
        texto_formatado += f"{chave}: {valor}\n"
    return texto_formatado

def menu_opcoes():
    root = tk.Tk()
    root.withdraw() # Esconde a janela principal do tkinter

    resposta = messagebox.askquestion("Opções", "Deseja analisar uma planta?")

    if resposta == 'yes':
        arquivo = selecionar_arquivo()

        genai.configure(api_key=API_KEY)

        img = PIL.Image.open(arquivo)

        model = genai.GenerativeModel('gemini-pro-vision')

        response = model.generate_content(['A imagem corresponde a uma planta? Qual é o nome comum da planta? Qual é a espécie da planta? Qual incidência solar é melhor para o seu desenvolvimento? Quantas vezes deve ser regada por semana? Qual a temperatura ideal para o cultivo? A planta da imagem apresenta algum problema de saúde? Estruture a resposta em formato JSON, seguindo o exemplo abaixo: { "Nome comum" : "Nome-comum", "Espécie" : "Specie_1 sp.", "Incidência solar" : "sol pleno", "Frequência de rega (semanal)" : "2", "Frequência de fertilização (mensal)" : "1", "Temperatura de cultivo" : "15-19ºC", "Condições da planta" : "saudável"}', img])
        dic = configura_json(response)

        if popup_opcoes(dic):
            main(dic)
        else:
            menu_opcoes()  # Chama a função novamente para retornar ao início
    else:
        pass

menu_opcoes()